# AWS SageMaker and nomic-embed-text-v1.5
This notebook demos how to embed your documents using nomic-embed-text-v1.5 running on AWS SageMaker.

In [ ]:
!pip install nomic

In [2]:
from nomic.aws.sagemaker import embed_text

## SageMaker Configuration
Specify the Sagemaker endpoint you created here and the region the endpoint is hosted in.

In [9]:
endpoint_name = 'nomic-embed-text-v-1-5-endpoint'
region_name = 'us-east-2'

## Real Time Inference
Run `embed_text` for real time inference.
The method will return a `np.float16 array`.

In [4]:
texts = [
    "This is a test", 
    "This is another test",
    "This is a third test",
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox jumps over the lazy dog",
]

In [10]:
embeddings = embed_text(texts, endpoint_name, region_name=region_name, batch_size=32)

In [11]:
embeddings.shape

(6, 768)